In [1]:
!pip install einops
!pip install fancy_einsum
!pip install jaxtyping
!pip install transformer_lens
!pip install sae_lens

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
from tqdm import tqdm
from huggingface_hub import login
import torch
import torch.nn as nn
import math
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px
from jaxtyping import Float
from functools import partial
import transformer_lens.utils as utils
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix
from collections import defaultdict

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-12-08 16:21:34.299485: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-08 16:21:34.308527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2

In [3]:
with open("datasets/access.tok", "r") as file:
    access_token = file.read()
    login(token=access_token)

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

# device = xm.xla_device()

print(f"Device: {device}")

Device: cuda


In [4]:
from datasets import load_dataset
import transformer_lens
from transformer_lens import HookedTransformer
from sae_lens import SAE, HookedSAETransformer

torch.set_grad_enabled(False)

NEG_SET_SIZE = 1000
POS_SET_SIZE = 1000

In [5]:
# load gemma model
model = HookedSAETransformer.from_pretrained(
    "meta-llama/Llama-3.1-8B", 
    device = device
)

num_layer = 16
# load sae on res stream of gemma model, plus cfg and sparsity val
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "llama_scope_lxr_8x",
    sae_id = f"l{num_layer}r_8x",
    device = device
)
# get layers from here: https://jbloomaus.github.io/SAELens/sae_table/#gemma-scope-2b-pt-res

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.28it/s]


Loaded pretrained model meta-llama/Llama-3.1-8B into HookedTransformer


In [6]:
neg_dataset = 'datasets/harmful_wiki_cleaned.csv'

df = pd.read_csv(neg_dataset)

columns_as_arrays = [df[col].values for col in df.columns]

array_dict = {col: df[col].values for col in df.columns}

negative_set = columns_as_arrays[0]
negative_set = negative_set[30:NEG_SET_SIZE+30]

negative_set_2 = columns_as_arrays[0]
negative_set_2 = negative_set_2[2030:NEG_SET_SIZE+2030]

print(len(negative_set))
print(len(negative_set_2))


1000
1000


In [7]:
#pos_dataset = 'dataset/alpaca_spanish.json'
pos_dataset = 'datasets/alpaca_data.json'

positive = pd.read_json(pos_dataset)

positive_set = positive['output'].values
positive_set = positive_set[:POS_SET_SIZE]
print(len(positive_set))

1000


In [8]:
sae.use_error_term

top_neurons_neg = defaultdict(list)
top_neurons_pos = defaultdict(list)

for example in negative_set:
    _, cache = model.run_with_cache_with_saes(example, saes=[sae])

    # get top 15 firing sae neurons
    vals, inds = torch.topk(cache[f'blocks.{num_layer}.hook_resid_post.hook_sae_acts_post'][0, -1, :], 15)

    for datapoint in zip(inds, vals):
        top_neurons_neg[int(datapoint[0])].append(datapoint[1].item())

    del cache


for example in positive_set:
    _, cache = model.run_with_cache_with_saes(example, saes=[sae])

    # get top 15 firing sae neurons
    vals, inds = torch.topk(cache[f'blocks.{num_layer}.hook_resid_post.hook_sae_acts_post'][0, -1, :], 15)
    for datapoint in zip(inds, vals):
        top_neurons_pos[int(datapoint[0])].append(datapoint[1].item())

    del cache

print(top_neurons_neg)
print(top_neurons_pos)

defaultdict(<class 'list'>, {6986: [10.0, 14.5, 8.25, 21.875, 12.8125, 21.75, 18.75, 19.25, 21.625, 18.25, 16.25, 18.75, 19.875, 20.5, 20.5, 22.5, 19.875, 19.125, 20.125, 17.25, 7.3125, 20.125, 25.625, 20.625, 21.75, 10.5625, 21.125, 15.6875, 7.3125, 12.0625, 13.25, 20.25, 13.0625, 24.25, 11.75, 14.125, 13.3125, 13.125, 9.3125, 26.75, 20.875, 11.9375, 23.5, 6.6875, 17.125, 15.75, 20.75, 9.625, 20.875, 21.75, 12.3125, 21.125, 21.75, 25.875, 17.75, 9.625, 18.0, 5.09375, 22.5, 17.375, 24.5, 11.25, 19.75, 11.0625, 13.1875, 13.0625, 21.125, 21.375, 18.5, 12.4375, 22.25, 16.625, 10.1875, 17.125, 18.25, 22.125, 10.75, 6.0, 23.0, 12.5625, 26.0, 19.625, 16.375, 24.375, 11.8125, 11.875, 18.375, 20.25, 21.625, 17.625, 26.0, 21.375, 10.25, 13.125, 13.8125, 18.0, 15.125, 27.625, 12.0, 9.3125, 17.75, 11.875, 19.5, 11.625, 24.625, 20.75, 15.75, 18.25, 19.5, 10.4375, 26.375, 15.5625, 21.25, 14.25, 15.0, 19.0, 21.5, 19.0, 22.5, 28.75, 12.875, 8.625, 13.375, 21.0, 22.5, 15.125, 19.875, 18.0, 21.75, 19.2

In [9]:
def filter_neurons(top_neurons_neg, top_neurons_pos, threshold=5.0):
    """
    Filters out neurons that are highly activated in both the negative and positive sets.
    """

    filtered_neurons_neg = {}
    filtered_neurons_pos = {}

    for neuron, activations in top_neurons_neg.items():
        if neuron in top_neurons_pos and any(val >= threshold for val in activations) and any(val >= threshold for val in top_neurons_pos[neuron]):
            continue
        else:
            filtered_neurons_neg[neuron] = activations

    for neuron, activations in top_neurons_pos.items():
        if neuron not in top_neurons_neg or not any(val >= threshold for val in top_neurons_neg[neuron]):
            filtered_neurons_pos[neuron] = activations

    return filtered_neurons_neg, filtered_neurons_pos

filtered_neg, filtered_pos = filter_neurons(top_neurons_neg, top_neurons_pos, 0)
print(f"Len: {len(filtered_neg)}. Filtered negative neurons: {filtered_neg}")
print(f"Len: {len(filtered_pos)}. Filtered positive neurons: {filtered_pos}")

Len: 1583. Filtered negative neurons: {15683: [9.75, 8.375, 6.78125, 6.03125, 7.125, 5.875, 5.6875], 1212: [7.5, 3.703125, 6.21875, 5.125], 8574: [7.28125], 9978: [5.65625, 3.96875, 3.9375], 20354: [5.125, 6.625, 6.90625, 8.0625], 14328: [5.03125, 10.5, 4.15625], 20182: [4.625, 3.359375, 2.96875, 3.640625], 23386: [4.5], 22045: [4.34375, 3.703125], 28887: [3.59375, 3.046875], 1061: [7.3125, 3.0], 20022: [4.5], 30307: [4.09375, 3.875, 3.671875, 3.84375, 5.0625, 2.765625], 18670: [3.578125, 10.3125, 3.796875, 3.796875, 6.96875, 4.125, 3.203125, 11.75, 6.5, 13.5625], 10608: [7.65625, 4.5625, 7.125, 8.5, 6.15625, 4.3125, 4.875, 4.6875, 5.65625, 4.40625, 8.0, 5.09375, 5.6875, 13.0, 3.96875, 12.1875, 6.375, 17.125, 5.875, 5.21875, 6.15625, 5.15625, 7.03125, 5.21875, 11.3125, 3.703125, 7.34375, 14.375, 9.625, 6.875, 4.84375, 3.9375, 8.625, 4.875, 6.375, 9.0625, 10.9375, 5.125, 13.75, 6.90625, 6.21875, 4.875, 7.875, 8.875, 6.65625, 8.4375, 9.8125, 8.5625, 4.3125, 3.5625, 15.0625, 12.9375, 6.84

In [10]:
# average activations over each top case, sends to
# top_neurons_neg/pos = {idx: avg_act, idx2:avg_act2, ...}
top_neurons_neg_mean = {}
for entry in filtered_neg:
    top_neurons_neg_mean[entry] = len(filtered_neg[entry])

top_neurons_pos_mean = {}
for entry in filtered_pos:
    top_neurons_pos_mean[entry] = len(filtered_pos[entry])

print(top_neurons_neg_mean)
print(top_neurons_pos_mean)

# sort by avg activation
top_neurons_neg_mean = {k: v for k, v in sorted(top_neurons_neg_mean.items(), key=lambda item: item[1], reverse=True)}
top_neurons_pos_mean = {k: v for k, v in sorted(top_neurons_pos_mean.items(), key=lambda item: item[1], reverse=True)}


# print first few
print(list(top_neurons_neg_mean.items())[:200])
print(list(top_neurons_pos_mean.items())[:200])

{15683: 7, 1212: 4, 8574: 1, 9978: 3, 20354: 4, 14328: 3, 20182: 4, 23386: 1, 22045: 2, 28887: 2, 1061: 2, 20022: 1, 30307: 6, 18670: 10, 10608: 106, 7597: 6, 18130: 21, 11194: 40, 20955: 10, 3157: 1, 31402: 8, 17644: 4, 19132: 10, 15564: 76, 10338: 313, 26883: 240, 2721: 17, 27208: 109, 26369: 23, 6288: 113, 5305: 122, 8315: 33, 28350: 46, 4646: 29, 4643: 45, 6230: 3, 31898: 1, 26657: 104, 12314: 1, 32178: 60, 3442: 3, 17224: 1, 31566: 3, 3305: 16, 30156: 1, 6728: 6, 26941: 6, 2046: 2, 17045: 8, 12667: 4, 30904: 25, 8395: 3, 17945: 29, 5931: 43, 26236: 2, 10172: 2, 28715: 2, 15015: 2, 6321: 92, 27295: 1, 32706: 1, 16731: 3, 8652: 2, 9979: 3, 16854: 1, 20613: 2, 7527: 23, 1920: 4, 25817: 6, 24179: 1, 3105: 1, 8664: 2, 5970: 2, 23035: 1, 7950: 1, 20854: 1, 14203: 1, 32165: 4, 12436: 9, 21: 3, 17030: 113, 24823: 4, 14304: 1, 9260: 1, 23831: 2, 27540: 2, 13170: 1, 6695: 11, 23417: 1, 7711: 2, 9377: 9, 15068: 19, 13103: 25, 32315: 6, 12758: 1, 18267: 1, 18289: 1, 32418: 1, 13659: 1, 1012: 

In [12]:
# train classifier on sae activations
activations_list = []
labels_list = []

# 0 = negative, 1 = positive
for example_txt in negative_set:
    _, cache = model.run_with_cache_with_saes(example_txt, saes=[sae])
    activations = cache[f'blocks.{num_layer}.hook_resid_post.hook_sae_acts_post'][0, -1, :].to(torch.float32).cpu().numpy()
    #print(activations.shape)

    del cache

    activations_list.append(activations)
    labels_list.append(0)

for example_txt in positive_set:
    _, cache = model.run_with_cache_with_saes(example_txt, saes=[sae])
    activations = cache[f'blocks.{num_layer}.hook_resid_post.hook_sae_acts_post'][0, -1, :].to(torch.float32).cpu().numpy()

    del cache

    activations_list.append(activations)
    labels_list.append(1)

# data
X = np.array(activations_list)
y = np.array(labels_list)

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale activation features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

clf = LogisticRegression(max_iter=1000, solver='lbfgs')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy:.4f}')
with open('run_log.txt', 'a') as file:
    file.write(f'SAE activations linear classifier accuracy on pos={pos_dataset} neg={neg_dataset} layer={num_layer} dataset_size={NEG_SET_SIZE}: {accuracy:.4f}\n')

Test Accuracy: 0.9825


In [13]:
# train classifier on base activations
activations_list = []
labels_list = []

# 0 = negative, 1 = positive
for example_txt in negative_set:
    _, cache = model.run_with_cache(example_txt)
    res_stream = cache.decompose_resid(layer=15, return_labels=False, mode='attn', incl_embeds=False, pos_slice=slice(-2, -1))
    seven_out = res_stream[-1, 0, -1, :].cpu().numpy() # layer batch pos d_model
    #print(seven_out.shape)

    del cache

    activations_list.append(seven_out)
    labels_list.append(0)

#print("done w bad")

for example_txt in positive_set:
    if (example_txt == ""):
        continue
    _, cache = model.run_with_cache(example_txt)
    res_stream = cache.decompose_resid(layer=15, return_labels=False, mode='attn', incl_embeds=False, pos_slice=slice(-2, -1))
    seven_out = res_stream[-1, 0, -1, :].cpu().numpy()  # layer batch pos d_model

    del cache

    activations_list.append(seven_out)
    labels_list.append(1)

# data and split
X = np.array(activations_list)
y = np.array(labels_list)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# train classifier
clf = LogisticRegression(max_iter=1000, solver='lbfgs')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy:.4f}')

print(f'Test Accuracy: {accuracy:.4f}')
with open('run_log.txt', 'a') as file:
    file.write(f'Base activations linear classifier accuracy on pos={pos_dataset} neg={neg_dataset} layer={num_layer} dataset_size={NEG_SET_SIZE}: {accuracy:.4f}\n')

Test Accuracy: 0.9675
Test Accuracy: 0.9675


In [17]:
# reject sample if activation at any of top N neurons is above threshold T
THRESHOLD = 1.75
NEURONS_TO_CONSIDER = 100

top_neurons_neg_occ = list(top_neurons_neg_mean.items())[:NEURONS_TO_CONSIDER]
top_neurons_pos_mean = []
neg_indicators = []

for entry in top_neurons_neg_occ:
    neg_indicators.append(entry[0])

print(neg_indicators)

# run an inference, if hits threshold on any of the neurons classify as bad
e = 0
neg_classifications = np.zeros(NEG_SET_SIZE)
for example in negative_set_2:
    _, cache = model.run_with_cache_with_saes(example, saes=[sae])

    # get data from indicator neurons
    val = cache[f'blocks.{num_layer}.hook_resid_post.hook_sae_acts_post'][0, -1, :]

    del cache

    for critical_neuron_idx in neg_indicators:
        if (val[critical_neuron_idx] > THRESHOLD):
            neg_classifications[e] = 1
            break

    e += 1

pos_classifications = np.zeros(POS_SET_SIZE)


e = 0
for example in positive_set:
    _, cache = model.run_with_cache_with_saes(example, saes=[sae])

    # get data from indicator neurons
    val = cache[f'blocks.{num_layer}.hook_resid_post.hook_sae_acts_post'][0, -1, :]

    del cache

    for critical_neuron_idx in neg_indicators:
        if (val[critical_neuron_idx] > THRESHOLD):
            pos_classifications[e] = 1
            break
    e += 1


print(" RATS ! ", np.sum(neg_classifications) / (NEG_SET_SIZE / 100))
print(" GENIUSES ! ", np.sum(pos_classifications) / (NEG_SET_SIZE / 100))

with open('run_log.txt', 'a') as file:
    file.write(f'SAE Thresholding true negative rate on pos={pos_dataset} neg={neg_dataset} layer={num_layer} dataset_size={NEG_SET_SIZE}: {np.sum(neg_classifications) / (NEG_SET_SIZE / 100)}; false negative rate: {np.sum(pos_classifications) / (NEG_SET_SIZE / 100)}\n')

[10338, 26883, 5305, 6288, 17030, 27208, 10608, 26657, 6321, 15564, 32178, 28350, 4643, 5931, 11194, 15938, 24202, 1012, 8315, 4007, 11931, 4646, 17945, 4443, 30904, 13103, 11374, 26369, 7527, 7883, 18130, 16654, 18115, 15068, 2721, 26441, 9991, 3305, 17787, 30714, 21731, 5737, 27100, 18861, 21080, 521, 14030, 13712, 12930, 6695, 14342, 18572, 17808, 228, 8505, 23642, 6498, 18670, 20955, 19132, 7338, 14381, 7240, 3404, 15656, 16809, 26345, 12436, 9377, 22751, 13009, 27632, 4909, 5455, 312, 17552, 31402, 17045, 1609, 20253, 8832, 21343, 7533, 14396, 6724, 17081, 16319, 30951, 15683, 32080, 25185, 24900, 15386, 26141, 26447, 15788, 15377, 29226, 8526, 14728]
 RATS !  98.1
 GENIUSES !  9.7


In [ ]:
from IPython.display import IFrame


html_template = "https://neuronpedia.org/{}/{}/{}?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300"

def get_dashboard_html(sae_release = "gemma-2-2b", sae_id="10-gemmascope-res-16k", feature_idx=0):
    return html_template.format(sae_release, sae_id, feature_idx)

for feature_idx in neg_indicators:
    html = get_dashboard_html(sae_release = "gemma-2-2b", sae_id="10-gemmascope-res-16k", feature_idx=feature_idx)
    frame = IFrame(html, width=800, height=400)
    display(frame)